<a href="https://colab.research.google.com/github/MegaVictinixyz/CS771-Course-Project/blob/main/task1_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***1. Import data and libraries***

In [ ]:
#Downloading dataset
!gdown https://drive.google.com/uc?id=16hB6UKRYC45BzAby92-eG19fLZ-rTqlA
!7z x dataset.zip

Downloading...
From (original): https://drive.google.com/uc?id=16hB6UKRYC45BzAby92-eG19fLZ-rTqlA
From (redirected): https://drive.google.com/uc?id=16hB6UKRYC45BzAby92-eG19fLZ-rTqlA&confirm=t&uuid=d535d75c-41c3-44de-a715-0c616651c907
To: /content/dataset.zip
100% 301M/301M [00:07<00:00, 41.9MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 301067282 bytes (288 MiB)

Extracting archive: dataset.zip
--
Path = dataset.zip
Type = zip
Physical Size = 301067282

  0%    
Would you like to replace the existing file:
  Path:     ./dataset/.DS_Store
  Size:     8196 bytes (9 KiB)
  Modified: 2024-10-29 16:26:33
with the file from archive:
  Path:     dataset/.DS_Store
  Size:     8196 bytes (9 KiB)
  Modified: 2024-10-29 16:26:33
? (Y)es / (N)o / (A)lways / (S)

In [ ]:
#Import all libraries used
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import mahalanobis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
#Load the data
eval=[]
for i in range(1,11):
  eval_path = '/content/dataset/part_one_dataset/eval_data/' + str(i) + '_eval_data.tar.pth'
  eval.append(torch.load(eval_path))
data=[]
for i in range(1,11):
  data_path = '/content/dataset/part_one_dataset/train_data/' + str(i) + '_train_data.tar.pth'
  data.append(torch.load(data_path))

<ipython-input-3-5013b179767f>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eval.append(torch.load(eval_path))
<ipython-input-3-5013b179767f>:9: FutureWarning: You are u

# ***2. Feature extraction and pre-processing***

In [ ]:
#Create separated list of datasets and labels
X = []
Y = []
XV = []
YV = []
for i in range(10):
    X.append(data[i]['data'])
    XV.append(eval[i]['data'])
    if(i==0):
      Y.append(data[i]['targets'])
    YV.append(eval[i]['targets'])

In [ ]:
# Load model directly
from transformers import AutoFeatureExtractor, AutoModel
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Load feature extractor and model
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Set model to evaluation mode
model.eval()

# Define transformations to resize images from X to 224x224
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

# Custom Dataset class for your image dataset X
class CustomImageDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx].astype(np.uint8)) # Assuming your image data is uint8 type
        if self.transform:
            image = self.transform(image)
        return image

# Function to extract features
def extract_features(data_loader, model):
    features = []
    with torch.no_grad():
        for images in data_loader:
            # Move images to device (GPU if available)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            images = images.to(device)

            # Extract features from the model
            outputs = model(images)
            # Use `outputs.last_hidden_state.mean(dim=1)` for mean pooled features
            # Or use `outputs.pooler_output` for [CLS] token representation
            features_batch = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            features.append(features_batch)

    # Combine features into a single numpy array
    features = np.vstack(features)
    return features

In [ ]:
#Extract
ext_feat = []
ext_feat_val = []

for dataset in X:
  batch_size = 32
  dataset = CustomImageDataset(dataset, transform=transform)
  data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
  features = extract_features(data_loader, model)
  ext_feat.append(features)

for dataset in XV:
  batch_size = 32
  dataset = CustomImageDataset(dataset, transform=transform)
  data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
  features = extract_features(data_loader, model)
  ext_feat_val.append(features)

In [ ]:
ext_feat[0].shape

(2500, 768)

In [ ]:
#Reshape in correct shape (2500,768)
X_feat=[0]*10
X_feat_val=[0]*10
for i in range(10):
   X_feat[i]=ext_feat[i].reshape(2500,-1)
   X_feat_val[i]=ext_feat_val[i].reshape(2500,-1)

In [ ]:
#Dimentionality reduction for data

#Perform LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_feat[0], Y[0])
for i in range(10):
  X_feat[i] = lda.transform(X_feat[i])
  X_feat_val[i] = lda.transform(X_feat_val[i])

In [ ]:
X_feat[0].shape

(2500, 9)

# ***3. Simple LwP with Mahalanobis distance***

In [ ]:
#Manhatten distance
#W = Inverce Covariance matrix of dataset 1
W = np.linalg.pinv(np.cov(X_feat[0].T))
def dist(x,y):
  return np.dot(np.dot((x-y).T,W),(x-y))

In [ ]:
#Class means of 1st dataset: Our F1
def create_f1(X,Y):
  class_means = []
  class_counts = []
  for class_label in range(10):
    class_indices = np.where(Y == class_label)[0]
    class_counts.append(len(class_indices))
    if len(class_indices) > 0:
      class_samples = X[class_indices]
      class_mean = np.mean(class_samples, axis=0)
      class_means.append(class_mean)

  class_means = np.array(class_means)
  return class_means, class_counts

In [ ]:
#Simple LwP and model updation
def simple_LwP(X_feat,Y,f1, class_counts):
  models=[]
  for i in range(10):
    models.append([0]*10)
  models[0] = f1

  predicted_labels=[]
  for i in range(10):
    predicted_labels.append([])
  predicted_labels[0]=Y[0]
  for i in range(9):
    for sample in X_feat[i+1]:
      distances = [dist(sample , mean) for mean in models[i]]
      predicted_label = np.argmin(distances)
      predicted_labels[i+1].append(predicted_label)

    for class_label in range(10):
      class_indices = [j for j in range(2500) if predicted_labels[i+1][j] == class_label]

      if len(class_indices) > 0:
        class_samples = X_feat[i+1][class_indices]
        class_mean = np.mean(class_samples, axis=0)
        models[i+1][class_label] = (class_mean * len(class_indices) + models[i][class_label] * class_counts[class_label] ) / ( len(class_indices) + class_counts[class_label] )

      class_counts[class_label]+=len(class_indices)

  return models

#Predict labels a dataset X using a model
def predict(model,X):
  predicted_labels = [0]*2500
  for i in range(2500):
    distances = [dist(X[i] , mean) for mean in model]
    label = np.argmin(distances)
    predicted_labels[i]=label
  return predicted_labels

In [ ]:
#Get first model and others via updation
f1, class_counts = create_f1(X_feat[0],Y[0])
all_models = simple_LwP(X_feat,Y,f1,class_counts)

In [ ]:
#Prediction Table for simple_LwP
table = [[0 for i in range(10)] for j in range(10)]
for i in range(10):
  for j in range(i+1):
    validation_accuracy = accuracy_score(YV[j], predict(all_models[i],X_feat_val[j]))
    table[i][j]=validation_accuracy*100

In [ ]:
#Print table
simple_LwP_table = pd.DataFrame(table)
simple_LwP_table.index = ['f'+str(i) for i in range(1,11)]
simple_LwP_table.columns = ['D'+str(i) for i in range(1,11)]
print(simple_LwP_table)

        D1     D2     D3     D4     D5     D6     D7     D8     D9   D10
f1   96.08   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.0
f2   96.04  96.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.0
f3   96.00  96.04  95.92   0.00   0.00   0.00   0.00   0.00   0.00   0.0
f4   95.96  96.04  95.96  95.56   0.00   0.00   0.00   0.00   0.00   0.0
f5   96.00  96.04  95.92  95.60  96.52   0.00   0.00   0.00   0.00   0.0
f6   96.00  96.04  95.92  95.60  96.52  96.40   0.00   0.00   0.00   0.0
f7   96.00  96.04  95.92  95.60  96.52  96.36  95.52   0.00   0.00   0.0
f8   95.92  96.04  95.92  95.60  96.56  96.36  95.52  95.68   0.00   0.0
f9   96.00  96.04  95.92  95.60  96.56  96.36  95.52  95.68  95.72   0.0
f10  95.92  96.04  95.92  95.56  96.56  96.36  95.52  95.68  95.72  96.2


# ***4. Gaussian Mixture Model***

In [ ]:
#Gaussian Mixture Model
def GMM(X,Y):
  #Returns distribution (mean, variance) and prior (class counts)
  class_means = []
  class_counts = []
  for class_label in range(10):
    class_indices = np.where(Y == class_label)[0]
    class_counts.append(len(class_indices))
    if len(class_indices) > 0:
      class_samples = X[class_indices]
      class_mean = np.mean(class_samples, axis=0)
      class_means.append(class_mean)

  class_means = np.array(class_means)

  COV=np.array([[[0.0 for j in range(len(X[0]))] for i in range(len(X[0]))] for k in range(10)])
  for i in range(2500):
    COV[Y[i]]+=np.outer(X[i]-class_means[Y[i]],X[i]-class_means[Y[i]])
  for k in range(10):
    COV[k]/=class_counts[k]

  return class_means,COV,class_counts

#Generate samples from learned distribution
def Generate_samples(recall,class_means,COV,class_counts,threshold):
  generated_samples = []
  N = sum(class_counts)
  for k in range(10):
    det = np.linalg.det(COV[k])
    inv = np.linalg.pinv(COV[k])
    i = 0
    while i<((recall*class_counts[k])//N):
      sample = np.random.multivariate_normal(class_means[k], COV[k])
      if np.log(class_counts[k]) - (0.5)*np.log(np.abs(det)) - (0.5)*((sample-class_means[k]).T @ (inv) @ (sample-class_means[k])) > threshold:
        generated_samples.append(sample)
        i+=1
  generated_samples = np.array(generated_samples)
  return generated_samples

#Predict on a dataset X
def predict_gmm(X,class_means,COV,class_counts):
  predicted_probs = [[0 for k in range(10)] for i in range(2500)]
  for k in range(10):
    det = np.linalg.det(COV[k])
    inv = np.linalg.pinv(COV[k])
    for i in range(2500):
      predicted_probs[i][k] = np.log(class_counts[k]) - (0.5)*np.log(np.abs(det)) - (0.5)*((X[i]-class_means[k]).T @ (inv) @ (X[i]-class_means[k]))

  predicted_labels = [0]*2500

  for i in range(2500):
    predicted_labels[i] = np.argmax(predicted_probs[i])

  predicted_labels = np.array(predicted_labels)
  return predicted_labels

In [ ]:
recall = 2500 #Recall for generating samples
threshold = -5 #For generating better samples (score >-5 only)

#Model creating and updation and prediction
class_means,COV,class_counts = GMM(X_feat[0],Y[0])
table = [[0 for i in range(10)] for j in range(10)]
table[0][0] = accuracy_score(YV[0], predict_gmm(X_feat_val[0],class_means,COV,class_counts))*100
for i in range(1,10):
  generated_samples = Generate_samples(recall*i,class_means,COV,class_counts,threshold)
  Union_data = np.concatenate((X_feat_val[i],generated_samples),axis=0)
  Union_labels = predict_gmm(Union_data,class_means,COV,class_counts)
  class_means,COV,class_counts = GMM(Union_data,Union_labels)
  for j in range(i+1):
    validation_accuracy = accuracy_score(YV[j], predict_gmm(X_feat_val[j],class_means,COV,class_counts))
    table[i][j]=validation_accuracy*100

In [ ]:
#Print table
GMM_table = pd.DataFrame(table)
GMM_table.index = ['f'+str(i) for i in range(1,11)]
GMM_table.columns = ['D'+str(i) for i in range(1,11)]
print(GMM_table)

        D1     D2     D3     D4     D5     D6     D7     D8     D9    D10
f1   95.84   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00
f2   95.72  96.12   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00
f3   95.04  95.56  95.48   0.00   0.00   0.00   0.00   0.00   0.00   0.00
f4   94.36  95.60  95.20  95.12   0.00   0.00   0.00   0.00   0.00   0.00
f5   93.76  95.48  94.84  94.56  95.16   0.00   0.00   0.00   0.00   0.00
f6   93.64  95.12  94.56  94.44  94.96  95.00   0.00   0.00   0.00   0.00
f7   93.88  94.96  94.64  94.28  94.76  94.56  95.12   0.00   0.00   0.00
f8   93.60  94.80  94.16  94.40  95.00  94.52  95.28  93.88   0.00   0.00
f9   93.40  94.72  94.16  94.24  94.60  94.52  94.84  93.60  93.88   0.00
f10  93.24  94.24  93.80  94.20  94.28  94.48  94.52  93.32  93.56  94.84
